# Forecasting USD-MNT Exchange Rate Using Machine Learning

In this analysis I hope to show that it is possible and practical to forecast USD-MNT exchange rates using machine learning. Traditional methods for forecasting exchange rates rely on complicated formulas or time series forecasting. With the advent of machine learning I hope to show that a reliable forecast can be made using new machine learning techniques. 

Macroeconomics literature states that there are many factors that contribute to determine the exchange rate of a currency pair. As many of these factors are included as possible given available statistical data. 

External factors to test:
- M1 and M2 money supplies
- Foreign currency on deposit
- Inflation rates
- Interest rates (policy rate)
- Balance of Payments

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('/storage/mds.mplstyle')

## Importing and Formatting Datasets

First we will import our various datasets. As they come in various time frequencies, we will resample each dataset to be monthly. We will handle each dataset as follows:

- For our exchange rate, we will take the ending price. 
- For monthly datasets no resampling is needed. 
- For quarterly datasets we use the quarterly value for each month in that period.

In [3]:
# Convenience function that 1) converts date field to datetime, 2) sets date field as index, 3) resamples to monthly
def convert_dataframe(df):
    '''Fields: df = dataframe, labelname = name of field label
    Requires that the date field be called "Date"'''
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)
    df = df.resample('1M').last()

### Exchange Rate

In [4]:
import os
os.listdir()

['Data',
 'USD-MNT Machine Learning - EDA.ipynb',
 'USD-MNT Machine Learning Model - Data Import and Formatting.ipynb',
 'tpot_exported_pipeline.py',
 'Output',
 '.ipynb_checkpoints',
 'AutoML_with_TPOT_for_Exchange_Rate_Forecasting.ipynb',
 'USD-MNT ARIMA Model - Abandoned.ipynb']

In [5]:
mnt = pd.read_csv('Data/Raw Data/USD-MNT.csv')

In [6]:
mnt.head()

,Date,USD
0,1/2/2008,1169.97
1,1/3/2008,1169.97
2,1/4/2008,1169.97
3,1/7/2008,1169.81
4,1/8/2008,1170.11


In [7]:
mnt.tail()

,Date,USD
3602,11/2/2018,2565.74
3603,11/3/2018,2565.74
3604,11/4/2018,2565.74
3605,11/5/2018,2565.90
3606,11/6/2018,2565.73


In [8]:
mnt.dtypes

Date     object
USD     float64
dtype: object

In [9]:
mnt['Date'] = pd.to_datetime(mnt['Date'])

In [10]:
mnt.set_index('Date', inplace=True)

In [11]:
mnt = mnt.resample('1M').last()

In [12]:
mnt.index

DatetimeIndex(['2008-01-31', '2008-02-29', '2008-03-31', '2008-04-30',
               '2008-05-31', '2008-06-30', '2008-07-31', '2008-08-31',
               '2008-09-30', '2008-10-31',
               ...
               '2018-02-28', '2018-03-31', '2018-04-30', '2018-05-31',
               '2018-06-30', '2018-07-31', '2018-08-31', '2018-09-30',
               '2018-10-31', '2018-11-30'],
              dtype='datetime64[ns]', name='Date', length=131, freq='M')

In [13]:
mnt = mnt[mnt.index <= '2018-11']

### CPI Percent Change

In [14]:
cpi = pd.read_csv('Data/Raw Data/DT_NSO_0600_009V1_-_2018-11-19 - CPI Change.csv')

In [15]:
cpi.head()

,INDICATOR,1991-10,1991-11,1991-12,1992-01,1992-02,1992-03,1992-04,1992-05,1992-06,...,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10
0,Overall index,3.2,5.1,4.4,3.6,9.8,32.9,13.7,4.5,24.0,...,1.4,0.8,0.9,0.6,0.6,0.5,0.5,-0.8,-0.1,1.0


In [16]:
cpi = cpi.melt(id_vars='INDICATOR', var_name='Date')

In [17]:
cpi.head()

,INDICATOR,Date,value
0,Overall index,1991-10,3.2
1,Overall index,1991-11,5.1
2,Overall index,1991-12,4.4
3,Overall index,1992-01,3.6
4,Overall index,1992-02,9.8


In [18]:
cpi['Date'] = pd.to_datetime(cpi['Date'])
cpi.set_index('Date', inplace=True)
cpi = cpi.drop(labels='INDICATOR', axis=1)
cpi = cpi.resample('1M').last()

In [19]:
cpi.head()

,value
Date,
1991-10-31,3.2
1991-11-30,5.1
1991-12-31,4.4
1992-01-31,3.6
1992-02-29,9.8


In [20]:
cpi.tail()

,value
Date,
2018-06-30,0.5
2018-07-31,0.5
2018-08-31,-0.8
2018-09-30,-0.1
2018-10-31,1.0


### Money Supply

In [21]:
money = pd.read_csv('Data/Raw Data/DT_NSO_0700_001V2_-_2018-11-19 - Money Supply.csv')

In [22]:
money.head()

,Indicators,1997-01,1997-02,1997-03,1997-04,1997-05,1997-06,1997-07,1997-08,1997-09,...,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10
0,Money M2,116.6,115.0,114.5,123.4,133.1,137.9,152.8,150.6,151.6,...,"15,763","15,912.4","16,073.9","16,240.1","17,045.3","17,342.9","17,189.3","17,440.8","17,911.8","18,311.5"
1,Money M1,57.1,56.9,52.7,59.3,64.5,67.6,75.4,74.1,75.7,...,"2,632.7","2,733.6","2,776.7","2,871","3,201.5","3,352.9","3,086","3,222.8","3,412.7","3,428.6"
2,Quasi money,59.5,58.1,61.8,64.1,68.6,70.2,77.5,76.5,75.9,...,"13,130.3","13,178.9","13,297.3","13,369.2","13,843.8","13,990","14,103.3","14,217.9","14,499.1","14,882.9"
3,Currency issued in circulation,44.7,43.8,41.1,45.1,48.8,52.8,57.3,58.0,58.2,...,801.9,850.4,840.4,937.6,935.9,931.1,917.1,927.5,929.1,887.8


In [23]:
money = money.melt(id_vars='Indicators', var_name='Date')

In [24]:
money.Indicators.unique()

array(['Money M2', 'Money M1', 'Quasi money',
       'Currency issued in  circulation'], dtype=object)

In [25]:
money.head()

,Indicators,Date,value
0,Money M2,1997-01,116.6
1,Money M1,1997-01,57.1
2,Quasi money,1997-01,59.5
3,Currency issued in circulation,1997-01,44.7
4,Money M2,1997-02,115


In [26]:
m2 = money[money['Indicators'] == 'Money M2']
m1 = money[money['Indicators'] == 'Money M1']

In [27]:
m2['Date'] = pd.to_datetime(m2['Date'])
m2.set_index('Date', inplace=True)
m2 = m2.resample('1M').last()
m2.drop(labels='Indicators', axis=1, inplace=True)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [28]:
m1['Date'] = pd.to_datetime(m1['Date'])
m1.set_index('Date', inplace=True)
m1 = m1.resample('1M').last()
m1.drop(labels='Indicators', axis=1, inplace=True)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


### Balance of Payments

The original data was modified in Excel to resample from quarter to months. 

In [29]:
balance = pd.read_csv('Data/Modified Data/Balance of Payments - Edited.csv')

In [30]:
balance.head()

,Classification,Date,Value
0,Current Account,1/1998,-8.8
1,Current Account,2/1998,-8.8
2,Current Account,3/1998,-8.8
3,Current Account,4/1998,-71.9
4,Current Account,5/1998,-71.9


In [31]:
balance.Classification.unique()

array(['Current Account', 'Capital and financial account',
       'Capital account', 'Financial account', 'Net Errors and Omissions'], dtype=object)

In [32]:
current = balance[balance['Classification'] == 'Current Account']
capital = balance[balance['Classification'] == 'Capital account']
financial = balance[balance['Classification'] == 'Financial account']
errors = balance[balance['Classification'] == 'Net Errors and Omissions']

In [33]:
current['Date'] = pd.to_datetime(current['Date'])
current.set_index('Date', inplace=True)
current = current.resample('1M').last()
current.drop(labels='Classification', axis=1, inplace=True)

capital['Date'] = pd.to_datetime(capital['Date'])
capital.set_index('Date', inplace=True)
capital = capital.resample('1M').last()
capital.drop(labels='Classification', axis=1, inplace=True)

financial['Date'] = pd.to_datetime(financial['Date'])
financial.set_index('Date', inplace=True)
financial = financial.resample('1M').last()
financial.drop(labels='Classification', axis=1, inplace=True)

errors['Date'] = pd.to_datetime(errors['Date'])
errors.set_index('Date', inplace=True)
errors = errors.resample('1M').last()
errors.drop(labels='Classification', axis=1, inplace=True)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/

In [34]:
current.head()

,Value
Date,
1998-01-31,-8.8
1998-02-28,-8.8
1998-03-31,-8.8
1998-04-30,-71.9
1998-05-31,-71.9


In [35]:
current.tail()

,Value
Date,
2018-02-28,-359.9
2018-03-31,-359.9
2018-04-30,-319.4
2018-05-31,-319.4
2018-06-30,-319.4


### External Trade

In [36]:
trade = pd.read_csv('Data/Raw Data/DT_NSO_1400_001V1_-_2018-11-19 - External Trade.csv')

In [37]:
trade.head()

,main indicators of foreign trade,1997-01,1997-02,1997-03,1997-04,1997-05,1997-06,1997-07,1997-08,1997-09,...,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10
0,Export,23.4,46.8,102.0,138.3,180.3,232.2,265.7,309.2,366.5,...,423.84,838.4,"1,482.41","2,125.40","2,885.20","3,581.40","4,116.70","4,671.20","5,282.80","5,944.80"
1,Imports,15.9,35.8,70.7,103.7,126.3,180.1,220.7,265.4,303.6,...,361.13,726.0,"1,130.47","1,581.90","2,178.80","2,759.10","3,265.70","3,802.00","4,351.10","4,875.60"
2,Balance,7.5,11.0,31.3,34.6,54.1,52.1,45.0,43.8,62.8,...,62.71,112.4,351.94,543.40,706.40,822.20,851.00,869.20,931.70,"1,069.20"


In [38]:
trade = trade.melt(id_vars='main indicators of foreign trade', var_name='Date')

In [39]:
balance = trade[trade['main indicators of foreign trade'] == 'Balance']

In [40]:
balance['Date'] = pd.to_datetime(balance['Date'])
balance.set_index('Date', inplace=True)
balance = balance.resample('1M').last()
balance.drop(labels='main indicators of foreign trade', axis=1, inplace=True)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


## Combine Dataframes

Not that we have the data imported and formatted the way we need it, we can combine the dataframes to start our analysis. The dataframes we need to combine are:

- mnt
- cpi
- m2
- m1
- quasi
- circulation
- domestic
- foreign
- current
- capital
- financial
- errors
- export
- imports
- balance
- government_balance

In [41]:
# Create df and merge MNT with CPI
df = mnt.merge(cpi, how='left', left_index=True, right_index=True)
df.rename(columns={'value': 'cpi'}, inplace=True)

In [42]:
# Merge df with m2
df = df.merge(m2, how='left', left_index=True, right_index=True)
df.rename(columns={'value': 'm2'}, inplace=True)

In [43]:
# Merge df with m1
df = df.merge(m1, how='left', left_index=True, right_index=True)
df.rename(columns={'value': 'm1'}, inplace=True)

In [44]:
# Merge df with current
df = df.merge(current, how='left', left_index=True, right_index=True)
df.rename(columns={'Value': 'current'}, inplace=True)

In [45]:
# Merge df with capital
df = df.merge(capital, how='left', left_index=True, right_index=True)
df.rename(columns={'Value': 'capital'}, inplace=True)

In [46]:
# Merge df with fiancial
df = df.merge(financial, how='left', left_index=True, right_index=True)
df.rename(columns={'Value': 'financial'}, inplace=True)

In [47]:
# Merge df with errors
df = df.merge(errors, how='left', left_index=True, right_index=True)
df.rename(columns={'Value': 'errors'}, inplace=True)

In [48]:
# Merge df with balance
df = df.merge(balance, how='left', left_index=True, right_index=True)
df.rename(columns={'value': 'balance'}, inplace=True)

In [50]:
df.head()

,USD,cpi,m2,m1,current,capital,financial,errors,balance
Date,,,,,,,,,
2008-01-31,1171.77,2.0,"2,307",551.9,-63.3,18.6,242.7,-147.4,17.6
2008-02-29,1171.82,2.7,"2,417.2",549.2,-63.3,18.6,242.7,-147.4,57.3
2008-03-31,1168.17,3.4,"2,454.8",577.7,-63.3,18.6,242.7,-147.4,-19.5
2008-04-30,1164.94,5.7,"2,466.3",583.7,-130.6,20.1,282.8,-161.4,-93.5
2008-05-31,1161.37,4.3,"2,516.2",620.6,-130.6,20.1,282.8,-161.4,-153.40


In [51]:
df.tail()

,USD,cpi,m2,m1,current,capital,financial,errors,balance
Date,,,,,,,,,
2018-06-30,2462.82,0.5,"17,342.9","3,352.9",-319.4,26.5,-376.6,-176.7,822.20
2018-07-31,2463.25,0.5,"17,189.3","3,086",NaN,NaN,NaN,NaN,851.00
2018-08-31,2472.99,-0.8,"17,440.8","3,222.8",NaN,NaN,NaN,NaN,869.20
2018-09-30,2552.13,-0.1,"17,911.8","3,412.7",NaN,NaN,NaN,NaN,931.70
2018-10-31,2564.44,1.0,"18,311.5","3,428.6",NaN,NaN,NaN,NaN,"1,069.20"


In [52]:
df.to_csv('Data/Modified Data/data.csv')